## Prototype for `napari` in a web-browser using `fastplotlib` as the front-end

In [21]:
import fastplotlib as fpl
import napari
import numpy as np
import imageio.v3 as iio
from ipywidgets import IntSlider, VBox
from functools import partial

### Create a 2D image in `napari` and view in `fastplotlib`

In [14]:
data = iio.imread('imageio:camera.png')

In [15]:
napari_img = napari.layers.Image(data=data)

In [16]:
napari_img.contrast_limits

[0, 255]

#### Simple `napari` 2D image_wrapper using `fastplotlib`

In [17]:
def image_wrapper(image: napari.layers.Image) -> fpl.graphics.ImageGraphic:
    """Return fastplotlib ImageGraphic based on napari Image layer.
    
    Parameters
    ==========
    image: napari.layers.Image
        napari Image layer to wrap
    
    Returns
    =======
    fpl.graphics.ImageGraphic
    """
    image_graphic = fpl.graphics.ImageGraphic(data=image.data, 
                        name=image.name, 
                        metadata=image.metadata,
                        cmap=image.colormap.name,
                        vmin=image.contrast_limits[0],
                        vmax=image.contrast_limits[1]
                       )
    
    # event handler to update contrast limits
    def update_vmin_vmax(ev):
        image_graphic.cmap.vmin, image_graphic.cmap.vmax = ev.source.contrast_limits
    
    # link napari contrast limits to update fpl plot
    image.events.contrast_limits.connect(update_vmin_vmax)
    
    return image_graphic

In [18]:
# create fastplotlib plot instance
plot = fpl.Plot()

# wrap napari image
fpl_img = image_wrapper(image=napari_img)

# add wrapped image to plot 
plot.add_graphic(fpl_img)

# view plot 
plot.show()

RFBOutputContext()

In [19]:
# flip image
plot.camera.world.scale_y *= -1

#### Update contrast limits of `napari` image

In [20]:
napari_img.contrast_limits = (10, 200)

### Create `fastplotlib` viewer that wraps `napari` viewer

In [27]:
class FplViewer(napari.components.ViewerModel):
    # napari uses pydantic, so must declare subclass specific attributes here
    plot: fpl.Plot = None
    sliders: list = None
    def __init__(
                self,
                *args,
                **kwargs):
        """
        Fastplotlib view wrapper to act as napari front-end viewer. Wraps the 
        napari ViewerModel class. 
        """
        super().__init__(**kwargs)
        
        self.plot = fpl.Plot()
        self.sliders = list()
        
        # connect inserting/removing layers to update fpl viewer
        self._connect_layer_events()
    
    def _add_graphic(self, ev):
        """
        On creation of a new napari layer, will parse the layer type and 
        add corresponding fastplotlib graphic instance. 
        """
        layer_insert_ix = ev.index
        if type(self.layers[layer_insert_ix]).__name__ == "Image":
            print("adding image")
            layer = self.layers[layer_insert_ix]
            # note: data being added is based on the current 2D slice being viewed
            self.plot.add_image(
                data=layer._slice.image.view,
                name=layer.name, 
                metadata=layer.metadata,
                cmap=layer.colormap.name,
                vmin=layer.contrast_limits[0],
                vmax=layer.contrast_limits[1],
            )
        else: # for now only have image implemented
            raise NotImplementedError
    
    def _remove_graphic(self, ev):
        """
        If napari layer removed, will remove corresponding graphic from fpl plot. 
        """
        layer_remove_ix = ev.index
        graphic = self.plot.graphics[layer_remove_ix]
        self.plot.remove_graphic(graphic)
        
    def _update_sliders(self):
        """
        Generate sliders for all dims not being displayed. 
        """
        self.sliders = list()
        
        dims_not_displayed = list(range(self.dims.ndim))[:-2]
        
        for dim in dims_not_displayed[::-1]:
            slider = IntSlider(value=int(self.dims.point[dim]),
                              min=self.dims.range[dim].start,
                              max=self.dims.range[dim].stop,
                              step=self.dims.range[dim].step)
            
            # update data based on dim slider values
            slider.observe(partial(self._update_point, dim), "value")
            self.sliders.append(slider)
        
    def _update_point(self, dim, change):
        """
        Event handler for changes in dim sliders. 
        """
        new_val = change["new"]
        self.dims.set_point(dim, new_val)
        for layer in self.layers:
            self.plot[layer.name].data = layer._slice.image.view
        
    
    def _connect_layer_events(self):
        """
        Connect napari layer events to fpl functions. 
        Must pass position='last' so that napari dims object will get updated
        first so data slice is correct. 
        """
        # insert layer, add graphic and update sliders
        self.layers.events.inserted.connect(self._add_graphic, position="last")
        self.layers.events.inserted.connect(self._update_sliders, position="last")
        # remove layer, remove graphic and update sliders
        self.layers.events.removed.connect(self._remove_graphic)
        self.layers.events.removed.connect(self._update_sliders)
    
    def show(self):
        """
        Return VBox of plot instance and sliders. 
        """
        return VBox([self.plot.show(),
                     *self.sliders]
                   )

In [28]:
data = np.random.rand(12, 16, 18, 22, 24)

In [29]:
img = napari.layers.Image(data=data)

In [30]:
fpl_view = FplViewer()

RFBOutputContext()

In [31]:
fpl_view.add_image(data)

adding image


/home/clewis7/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


<Image layer 'data' at 0x7fd50c33e090>

In [32]:
fpl_view.show()